In [1]:
import pandas as pd
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
import re
import requests

In [2]:
import requests

url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/hi/history/cobr.l/1d"

headers = {
    'x-rapidapi-key': "9cd97ba3camsh244cbd2e3e420a4p1e83c0jsne0fdd0787f8e",
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

In [3]:
string = response.text

In [20]:
string[:1000]

'{\n    "meta": {\n        "currency": "GBp",\n        "symbol": "COBR.L",\n        "exchangeName": "LSE",\n        "instrumentType": "EQUITY",\n        "firstTradeDate": 1542268800,\n        "regularMarketTime": 1628081844,\n        "gmtoffset": 3600,\n        "timezone": "BST",\n        "exchangeTimezoneName": "Europe/London",\n        "regularMarketPrice": 1.736,\n        "chartPreviousClose": 1.75,\n        "priceHint": 4,\n        "dataGranularity": "1d",\n        "range": ""\n    },\n    "items": {\n        "1542268800": {\n            "date": "15-11-2018",\n            "open": 1.67,\n            "high": 1.75,\n            "low": 1.67,\n            "close": 1.75,\n            "adjclose": 1.75\n        },\n        "1542355200": {\n            "date": "16-11-2018",\n            "open": 1.75,\n            "high": 1.75,\n            "low": 1.75,\n            "close": 1.75,\n            "adjclose": 1.75\n        },\n        "1542614400": {\n            "date": "19-11-2018",\n         

- 获取日期data

In [4]:
def get_dates(string):
    date = re.compile('"date":((.*),)')
    date = date.findall(string)
    date = [i[0][2:12] for i in date]
    date = [datetime.strptime(i, "%d-%m-%Y").strftime("%Y-%m-%d") for i in date]
    return date

In [10]:
date = get_dates(string)
print( date[:5] )

['2018-11-15', '2018-11-16', '2018-11-19', '2018-11-20', '2018-11-21']


- 获取close价格

In [14]:
def get_close(string):
    close = re.compile('"close":((.*),)')
    close = close.findall(string)
    close = [float(i[1].strip()) for i in close]
    return close

In [15]:
close = get_close(string)

In [18]:
close[:5]

[1.75, 1.75, 1.65, 1.65, 1.65]

- form dataframe

In [33]:
def form_df(date, close, col_name1, col_name2):
    return pd.DataFrame({col_name1:date, col_name2:close })

In [43]:
cobr = form_df(date, close, 'date', 'cobr_close')
cobr.set_index('date', drop=True, inplace=True)
cobr.tail(3)

,cobr_close
date,
2021-08-02,1.85
2021-08-03,1.85
2021-08-04,1.74


- save dataframe as csv

In [38]:
# cobr.to_csv(r'/Users/paxton615/Desktop/COBR.csv')

- IPO and Placing info

In [49]:
p1_sprice = 2.25
p1_wprice = 3
p1_date = '2020-01-01'
p1_swratio = 1/1
p2_sprice = 2.3
p2_wprice = 3
p2_date = '2020-09-01'
p2_swratio = 1/1

In [46]:
cobr['p1_sprice'] = [p1_sprice if i  >= p1_date else 0 for i in cobr.index]
cobr['p1_wprice'] = [p1_wprice if i  >= p1_date else 0 for i in cobr.index]

cobr['p2_sprice'] = [p2_sprice if i  >= p2_date else 0 for i in cobr.index]
cobr['p2_wprice'] = [p2_wprice if i  >= p2_date else 0 for i in cobr.index]

- If each Placing or IPO investor invest 10000 pounds ever time, what are the positions look like？

In [52]:
fund = 10000

In [56]:
cobr['p1_sposition'] = [round(fund/p1_sprice,2) if i >= p1_date else 0 for i in cobr.index]
cobr['p1_wposition'] = cobr['p1_sposition']*p1_swratio

cobr['p2_sposition'] = [round(fund/p2_sprice,2) if i >= p2_date else 0 for i in cobr.index]
cobr['p2_wposition'] = cobr['p2_sposition']*p2_swratio

- What is the profit looks like?

In [60]:
# Profit from the shares

cobr['p1_sprft'] = cobr.cobr_close - cobr['p1_sprice']

cobr['p2_sprft'] = cobr.cobr_close - cobr['p2_sprice']

In [61]:
# Profit from the warranties

p1_wprofit = []
for i,j in zip(cobr.cobr_close, cobr['p1_wprice']):
    if i - j >0:
        p1_wprofit.append(round(i-j,2))
    else:
        p1_wprofit.append(0)

cobr['p1_wprft'] = p1_wprofit

p2_wprofit = []
for i,j in zip(cobr.cobr_close, cobr['p2_wprice']):
    if i - j >0:
        p2_wprofit.append(round(i-j,2))
    else:
        p2_wprofit.append(0)

cobr['p2_wprft'] = p2_wprofit

In [63]:
cobr

,cobr_close,p1_sprice,p1_wprice,p2_sprice,p2_wprice,p1_sposition,p1_wposition,p2_sposition,p2_wposition,p1_sprft,p2_sprft,p1_wprft,p2_wprft
date,,,,,,,,,,,,,
2018-11-15,1.75,0.00,0,0.0,0,0.00,0.00,0.00,0.00,1.75,1.75,1.75,1.75
2018-11-16,1.75,0.00,0,0.0,0,0.00,0.00,0.00,0.00,1.75,1.75,1.75,1.75
2018-11-19,1.65,0.00,0,0.0,0,0.00,0.00,0.00,0.00,1.65,1.65,1.65,1.65
2018-11-20,1.65,0.00,0,0.0,0,0.00,0.00,0.00,0.00,1.65,1.65,1.65,1.65
2018-11-21,1.65,0.00,0,0.0,0,0.00,0.00,0.00,0.00,1.65,1.65,1.65,1.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-29,1.85,2.25,3,2.3,3,4444.44,4444.44,4347.83,4347.83,-0.40,-0.45,0.00,0.00
2021-07-30,1.80,2.25,3,2.3,3,4444.44,4444.44,4347.83,4347.83,-0.45,-0.50,0.00,0.00
2021-08-02,1.85,2.25,3,2.3,3,4444.44,4444.44,4347.83,4347.83,-0.40,-0.45,0.00,0.00
